# 学習状況のチェック (mp4)

走行動画を用いて推論の結果を表示する

In [ ]:
TASK = 'off001'

In [ ]:
TARGET = 'slowdrive'

input_video = 'movie/' + TARGET + '.mp4'
output_video = 'log/'  + TARGET + '_result.mp4'

In [ ]:
import torch
import torchvision

MODEL = TASK+"_model.pth"

if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'
    
device = torch.device(DEVICE)
output_dim = 2

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

model = model.to(device)
model.load_state_dict(torch.load('data/'+TASK+'_A/'+MODEL))

In [ ]:
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

if DEVICE== 'cuda':
    mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
    std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
else:
    mean = torch.Tensor([0.485, 0.456, 0.406]).cpu()
    std = torch.Tensor([0.229, 0.224, 0.225]).cpu()   


def preprocess(image):
    device = torch.device(DEVICE)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
import cv2                         # openCV
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import re
import ipywidgets as widgets
from ipywidgets import Textarea,Layout

IMG_WIDTH = 224
IMG_HEIGHT = 224
%matplotlib notebook

In [ ]:
video = cv2.VideoCapture(input_video) # 引数がファイルパス

width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH)) # 動画の画面横幅
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 動画の画面縦幅
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) # 総フレーム数
frame_rate = int(video.get(cv2.CAP_PROP_FPS)) # フレームレート(fps)
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') # ファイル形式(ここではmp4)
writer = cv2.VideoWriter(output_video, fmt, frame_rate, (IMG_WIDTH,IMG_HEIGHT))  # 射影変換後の画像サイズ

print('start processing ', input_video,' ...')
for i in range(frame_count):
    ret,img_tmp = video.read()
    img = cv2.resize(img_tmp, dsize=(224,224))
        
    preprocessed = preprocess(image)
    output = model(preprocessed).detach().cpu().numpy().flatten()
    result_x = output[0]
    result_y = output[1]
    result_x = int(IMG_WIDTH * (result_x / 2.0 + 0.5))
    result_y = int(IMG_HEIGHT * (result_y / 2.0 + 0.5))
    
    img = cv2.line(img,(112,0),(112,224),(255,255,255),3)
    img = cv2.line(img,( 77,0),( 77,224),(255,255,255),1)
    img = cv2.line(img,( 42,0),( 42,224),(255,255,255),1)
    img = cv2.line(img,(147,0),(147,224),(255,255,255),1)
    img = cv2.line(img,(182,0),(182,224),(255,255,255),1)
    img = cv2.line(img,(0, 75),(224, 75),(255,255,255),1)
    img = cv2.line(img,(0,112),(224,112),(255,255,255),1)
    img = cv2.line(img,(0,150),(224,150),(255,255,255),1)
    
    image = cv2.circle(img, (result_x,result_y), 10, (0, 0, 255),3)
    writer.write(image)
    
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    
    #plt.imshow(image)
    #plt.show()

video.release()
writer.release() 

print('result file:', output_video)